## Data Preparation Notebook

## Imports

In [1]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,LabelEncoder,MinMaxScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from botocore.client import Config
from ibm_watson_studio_pipelines import WSPipelines
import matplotlib.pyplot as plt

# import itc_utils.flight_service as itcfs

import heapq
import pickle
import os, types
import pandas as pd
import ibm_boto3

### Load the validated training and test data from IBM Filesystem (alternatively Db2) 

```

```

## Pipeline Params

In [2]:
TOKEN = os.getenv("USER_ACCESS_TOKEN")

In [3]:
train_data_filename = os.getenv("train_data_filename")
test_data_filename = os.getenv("test_data_filename")
path = os.getenv("path")

train_data_path = os.path.join(path, train_data_filename)
test_data_path = os.path.join(path, test_data_filename)

In [4]:
pipeline_filename = os.getenv("pipeline_filename")
if pipeline_filename is None:
    pipeline_filename = "feature_encode.pkl"

In [ ]:
# train_data_path = "mlops-dir/train_gcr.csv"
# test_data_path = "mlops-dir/test_gcr.csv"

###  Read and Write Utility

In [6]:
def save_data_in_filesystem(df,filename):
    """
    Save Data in Filesystem

    Passed filename should involve path

    """
    try:
        if filename[-3:] == "csv":
            df.to_csv(filename,index=False)
            print(f"File {filename} persisted successfully as csv")
        else:
            with open(filename, 'wb') as f:
                pickle.dump(df, f)
            print(f"File {filename} pickled successfully")
    except Exception as e:
        print(e)
        print(f"File serialization for {filename} failed")

def check_for_file_in_filesystem(path):
    """
    Check existence of path in filesystem
    """
    if os.path.exists(path):
        return True
    else:
        return False
    
def read_data_from_db2(data_request):
    read_client = itcfs.get_flight_client()
    DB2_DATA_data_request = {
        'connection_name': """DB2_DATA""",
        'interaction_properties': {
            'select_statement': 'SELECT * FROM "CUSTOMER_DATA"."GERMAN_CREDIT_RISK_TRAINING" FETCH FIRST 5000 ROWS ONLY'
        }
    }

    flightInfo = itcfs.get_flight_info(read_client, nb_data_request=data_request)

    df = itcfs.read_pandas_and_concat(read_client, flightInfo, timeout=240)
    return df
    
def load_data_from_filesystem(path):
    """
    Check existence of path in filesystem.
    If it does exist, loads csv via path
    If it does NOT exist, try to load data from Db2
    """
    body = check_for_file_in_filesystem(path)
    if body:
        suffix = path[-3:]
        # Check whether path ends on csv
        if suffix == "csv":
            gcf_df = pd.read_csv(path)
        else:
            with open(path) as f:
                gcf_df = pickle.load(f)

        return gcf_df
    else:
        print("\n")
        print(f"{path} file/path is probably not in project. Loading File from MLOps COS Bucket.")

        data_request = {
                'connection_name': """DB2_DATA""",
                'interaction_properties': {
                    'select_statement': 'SELECT * FROM "CUSTOMER_DATA"."GERMAN_CREDIT_RISK_TRAINING" FETCH FIRST 5000 ROWS ONLY'
                }
            }

        gcf_df = read_data_from_db2(data_request)
        return gcf_df

## Train_Data 

In [7]:
train_data = load_data_from_filesystem(train_data_path)
train_data.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,0
1,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,0
2,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,1
3,no_checking,9,prior_payments_delayed,car_new,1032,100_to_500,4_to_7,3,male,none,...,savings_insurance,41,none,own,1,management_self-employed,1,none,yes,0
4,0_to_200,11,credits_paid_to_date,car_new,4553,less_100,less_1,3,female,none,...,savings_insurance,22,none,own,1,management_self-employed,1,none,yes,0


In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3995 entries, 0 to 3994
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CheckingStatus            3995 non-null   object
 1   LoanDuration              3995 non-null   int64 
 2   CreditHistory             3995 non-null   object
 3   LoanPurpose               3995 non-null   object
 4   LoanAmount                3995 non-null   int64 
 5   ExistingSavings           3995 non-null   object
 6   EmploymentDuration        3995 non-null   object
 7   InstallmentPercent        3995 non-null   int64 
 8   Sex                       3995 non-null   object
 9   OthersOnLoan              3995 non-null   object
 10  CurrentResidenceDuration  3995 non-null   int64 
 11  OwnsProperty              3995 non-null   object
 12  Age                       3995 non-null   int64 
 13  InstallmentPlans          3995 non-null   object
 14  Housing                 

In [9]:
train_data.describe()

,LoanDuration,LoanAmount,InstallmentPercent,CurrentResidenceDuration,Age,ExistingCreditsCount,Dependents,Risk
count,3995.000000,3995.000000,3995.000000,3995.00000,3995.000000,3995.000000,3995.00000,3995.000000
mean,21.395745,3488.971464,2.977222,2.84781,35.979224,1.472340,1.16821,0.330914
std,11.175001,2500.504918,1.130774,1.12105,10.678289,0.567809,0.37410,0.470601
min,4.000000,250.000000,1.000000,1.00000,19.000000,1.000000,1.00000,0.000000
25%,12.500000,1328.000000,2.000000,2.00000,28.000000,1.000000,1.00000,0.000000
50%,21.000000,3237.000000,3.000000,3.00000,36.000000,1.000000,1.00000,0.000000
75%,29.000000,5334.500000,4.000000,4.00000,44.000000,2.000000,1.00000,1.000000
max,64.000000,11676.000000,6.000000,6.00000,74.000000,4.000000,2.00000,1.000000


In [10]:
object_df = train_data.select_dtypes('O')
object_df.head()

,CheckingStatus,CreditHistory,LoanPurpose,ExistingSavings,EmploymentDuration,Sex,OthersOnLoan,OwnsProperty,InstallmentPlans,Housing,Job,Telephone,ForeignWorker
0,less_0,credits_paid_to_date,car_new,less_100,1_to_4,female,none,savings_insurance,stores,own,skilled,none,yes
1,less_0,prior_payments_delayed,furniture,less_100,1_to_4,male,none,real_estate,none,own,skilled,yes,no
2,no_checking,prior_payments_delayed,education,500_to_1000,greater_7,male,none,unknown,none,own,skilled,none,yes
3,no_checking,prior_payments_delayed,car_new,100_to_500,4_to_7,male,none,savings_insurance,none,own,management_self-employed,none,yes
4,0_to_200,credits_paid_to_date,car_new,less_100,less_1,female,none,savings_insurance,none,own,management_self-employed,none,yes


In [11]:
object_cols = list(set(object_df.columns.tolist()) - set(['Risk']))
object_cols

['ForeignWorker',
 'Telephone',
 'OthersOnLoan',
 'ExistingSavings',
 'Job',
 'LoanPurpose',
 'InstallmentPlans',
 'Sex',
 'CheckingStatus',
 'CreditHistory',
 'OwnsProperty',
 'Housing',
 'EmploymentDuration']

In [12]:
numerical_columns = [col for col in train_data.columns.tolist() if col not in object_cols and col!='Risk']

## Load Test Data 

In [13]:
test_data = load_data_from_filesystem(test_data_path)
test_data.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,0
1,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,0
2,no_checking,32,outstanding_credit,vacation,9604,500_to_1000,greater_7,6,male,co-applicant,...,unknown,57,none,free,2,skilled,2,yes,yes,1
3,less_0,16,credits_paid_to_date,vacation,3109,less_100,4_to_7,3,female,none,...,car_other,36,none,own,2,skilled,1,none,yes,0
4,0_to_200,15,prior_payments_delayed,furniture,250,500_to_1000,4_to_7,3,male,none,...,savings_insurance,24,none,own,2,skilled,2,yes,yes,0


## Split X and Y 

In [ ]:
y_train = train_data['Risk']
X_train = train_data.drop("Risk",axis=1)


y_test = test_data['Risk']
X_test = test_data.drop("Risk",axis=1)

## Categorcial Feature Analysis 

In [15]:
def prepare_input_data(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc


def prepare_output_data(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc


def select_best_chi2_features(X_train, y_train, X_test,score_func=chi2):
    featureselector = SelectKBest(score_func=chi2, k='all')
    featureselector.fit(X_train, y_train)
    X_train_best_feat = featureselector.transform(X_train)
    X_test_best_feat= featureselector.transform(X_test)
    return X_train_best_feat, X_test_best_feat, featureselector


def select_best_mutualinf_features(X_train, y_train, X_test,k=5):
    featureselector = SelectKBest(score_func=mutual_info_classif, k=k)
    featureselector.fit(X_train, y_train)
    X_train_best_feat = fs.transform(X_train)
    X_test_best_feat= fs.transform(X_test)
    return X_train_best_feat, X_test_best_feat, featureselector

# def plot_scores():
#     plt.figure(figsize=(14, 12))
#     plt.subplot(221)

#     ax1 = sns.barplot([i for i in range(len(fs.scores_))], fs.scores_)
#     ax1.set_title("Chi2 Importance Scores", fontsize=20)
#     ax1.set_xlabel("Features",fontsize=15)
#     ax1.set_ylabel("Chi2 Scores",fontsize=15)
    
    
def get_top_k_catgeorical(fs,train_cat,k=10):
    fs_score_map = {}
    for i in range(len(fs.scores_)):
        #print(f"Feature {train_cat.columns.tolist()[i]} {fs.scores_[i]}")
        fs_score_map[train_cat.columns.tolist()[i]] = fs.scores_[i]
        
    k_keys_sorted_by_values = heapq.nlargest(k, fs_score_map, key=fs_score_map.get)
    
    return k_keys_sorted_by_values

## Encode and shape the Variables 

In [16]:
X_train_enc, X_test_enc = prepare_input_data(X_train[object_cols], X_test[object_cols])

y_train_enc, y_test_enc = prepare_output_data(y_train, y_test)

X_train_fs, X_test_fs, fs = select_best_chi2_features(X_train_enc, y_train_enc, X_test_enc)

# plot_scores()


## Top K Categorical Features  based on Chi2

In [17]:
top_k_cat = get_top_k_catgeorical(fs,X_train[object_cols])
top_k_cat

['CreditHistory',
 'Telephone',
 'CheckingStatus',
 'Housing',
 'OthersOnLoan',
 'OwnsProperty',
 'LoanPurpose',
 'ExistingSavings',
 'EmploymentDuration',
 'Job']

## Top K Categorical Features  based on Mutual Information Feature Selection

In [18]:
X_train_enc_mf, X_test_enc_mf = prepare_input_data(X_train[object_cols], X_test[object_cols])

y_train_enc_mf, y_test_enc_mf = prepare_output_data(y_train, y_test)

X_train_fs_mf, X_test_fs_mf, fs_mf = select_best_chi2_features(X_train_enc_mf, y_train_enc_mf, X_test_enc_mf)

# plot_scores()

In [19]:
top_k_cat_mf = get_top_k_catgeorical(fs_mf,X_train[object_cols])
top_k_cat_mf

['CreditHistory',
 'Telephone',
 'CheckingStatus',
 'Housing',
 'OthersOnLoan',
 'OwnsProperty',
 'LoanPurpose',
 'ExistingSavings',
 'EmploymentDuration',
 'Job']

In [20]:
union_features = list(set(top_k_cat+top_k_cat_mf))
if "Sex" not in union_features:
    union_features.append("Sex")
union_features

['Telephone',
 'OthersOnLoan',
 'ExistingSavings',
 'Job',
 'LoanPurpose',
 'CheckingStatus',
 'CreditHistory',
 'OwnsProperty',
 'Housing',
 'EmploymentDuration',
 'Sex']

## Filter the Top K Categorical features and Merge to Original Train and Test Dataframes

In [21]:
X_train_object_filtered = X_train[union_features]
X_test_object_filtered = X_test[union_features]

X_train_final = pd.concat([X_train[numerical_columns],X_train_object_filtered],axis=1)

X_test_final = pd.concat([X_test[numerical_columns],X_test_object_filtered],axis=1)

## Use Column Transformer and Pipelines to encode the Input and Output Variables . Scale the Numerical columns using MinMaxScaler.

In [22]:
numerical_ix = X_train_final.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X_train_final.select_dtypes(include=['object', 'bool']).columns

In [ ]:
encoding_steps = [('cat', OrdinalEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=encoding_steps)

In [24]:
pipeline = Pipeline(steps=[('prep',col_transform)])

In [25]:
train_final = pd.concat([X_train_final,y_train],axis=1)

In [26]:
test_final = pd.concat([X_test_final,y_test],axis=1)

In [27]:
#encoded_train = pd.DataFrame(pipeline.fit_transform(X_train_final),columns=X_train_final.columns)

In [28]:
#encoded_test = pd.DataFrame(pipeline.transform(X_test_final),columns=X_test_final.columns)

## Save the Prepared Data to IBM COS

In [29]:
save_data_in_filesystem(df=train_final, filename=train_data_path)

File mlops-dir/train_gcr.csv persisted successfully as csv


In [30]:
save_data_in_filesystem(df=test_final, filename=test_data_path)

File mlops-dir/test_gcr.csv persisted successfully as csv


In [31]:
pipeline_path = os.path.join(path, pipeline_filename)
save_data_in_filesystem(df=pipeline, filename=pipeline_path)

File feature_encode.pickle pickled successfully


## Check if files have been copied 

In [ ]:
data_prep_done = check_for_file_in_filesystem(train_data_path) and check_for_file_in_filesystem(test_data_path) and check_for_file_in_filesystem(pipeline_path)
data_prep_done

## Store Params in WS Pipelines

In [34]:
preparation_params = {}
preparation_params['data_prep_done'] = data_prep_done
preparation_params['pipeline_path'] = pipeline_filename

pipelines_client = WSPipelines.from_token(TOKEN)
pipelines_client.store_results(preparation_params)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "data_prep_done": .ibm_watson_studio_pipelines/results/data_prep_done
    - "pipeline_path": .ibm_watson_studio_pipelines/results/pipeline_path


/opt/conda/envs/Python-3.10-Premium/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc.cp4d.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
